In [85]:
import math
import numpy as np
from scipy.optimize import fsolve
from ambiance import Atmosphere
import matplotlib.pyplot as plt
import json

# Defining atmospheric conditions with a function
def ambient_props(altitude_meters):
    atmosphere = Atmosphere(altitude_meters)
    rho = atmosphere.density
    nu = atmosphere.kinematic_viscosity
    T = atmosphere.temperature
    g = atmosphere.grav_accel
    return rho, nu, T, g

# Import all the variables
altitude = 0 # ASL in metres 
altitide_cruise = 4000 # ASL in metres 
## Do not change the following ______________________:)
air = ambient_props(altitude)
rho, nu ,  T, g = air[0], air[1], air[2], air[3]
## __________________________________________________:)
# Read JSON
with open("reports\wing_sizing_res.json", "r") as f:
    data = json.load(f)

In [86]:
# Global variables Horizontal tail from guess all need to be honored
pi=math.pi

# Data from wing sizing
C_REF= float(data["C_tip"])        # Mean geometric chord
S_REF= float(data["S_req"])        # wing reference area
b_wing = float(data["b_req"])      # meters
AR = float(data["AR"])             # estimated from Great shark F320
alpha_0 = 0.000001
W_s = float(data["W_S"])
V_c = float(data["V"])             # Airspeed Cruise
V_s = float(data["V_s"])       # Stall Speed
V_h = V_c                    # effective airspeed at the horizontal tail region

#--------------------------------------------------------------------------------#
# The following values need to be  updated manually
alpha_f=0.0001              # update this value if wing design is changed
alpha_w= 1.9                # degrees update this value if the wing is installed at an angle
#--------------------------------------------------------------------------------#

V_H = 0.5                   # from table 6.4 Needs to be calculted
V_V = 0.033                 # from table 6.4 = 0.04 but from very similar aircraft
D_f = 0.2                   # estimated from Great shark F320
K_C = 1                     # ranges from 1 to 1.4 its a conenction factor page 300 pdf
Sweep= 0                    # 8 degress estimated from Great shark F320

C_mo_wf = 0                 # wing pitching moment coefficient
C_m_af = -0.013             # airfoil section pitching moment coefficient
alpha_t= 0.0000             # twist angle


etha_h = 0.98               # Goes from 0.85 to 0.95 page 276 book

# Parameters  Used to calculate the horizontal tail desired lift coefficient at cruise REVISE ALL!!!
h = 0                       # aircraft center of gravity
l_L = 0.45                  # lateral trim coefficient from table 6.2
h_o= 0.25                   # The aerodynamic center of the wing/fuselage combination is located at 
                            # 23% of MAC
#cg_p = 0.32 # aircraft center of gravity is located at 32% of the fuselage length
cg_d = 0.033                #!!! Assuming that the aircraft cg is 3.3 cm ahead of the wing/fuselage 
                            #aerodynamic center. I INVENTED THIS VALUE !!!!!
error_tolerance = 0.001     # Tolerance for the error
N = 9                       # (number of segments-1)

Horizontal Tail sizing
1. Select horizontal tail volume coefficient, V_H (Table 6.4).
2. Calculate optimum tail moment arm (l_opt) to minimize the aircraft drag and weight (Section 6.6).
3. Calculate horizontal tail planform area, S_h (Equation (6.24)).
6. Calculate wing/fuselage aerodynamic pitching moment coefficient C_mo_wf (Equation (6.26)).



In [87]:
L_opt = K_C*(np.sqrt((4*C_REF*S_REF*V_H)/(pi*D_f)))
print("Optimum tail moment arm =", L_opt)
S_h= (V_H*S_REF*C_REF)/(L_opt)
print("Tail plataform area =", S_h)
C_mo_wf = C_m_af*((AR*((np.cos((math.radians(Sweep))))*(np.cos((math.radians(Sweep))))))/(AR+(2*(np.cos((math.radians(Sweep)))))))+(0.01*alpha_t) # from xflr5 at cruise 0aoa wing gives -0.368
print("Wing ptching moment coefficient =", C_mo_wf)


Optimum tail moment arm = 0.7767926624417907
Tail plataform area = 0.12201830608447928
Wing ptching moment coefficient = -0.010636363636363637


7. Calculate cruise lift coefficient, C_L_c (Equation (6.27)).
8. Calculate horizontal tail desired lift coefficient at cruise from trim (Equation (6.29)).

In [88]:
rho_ac = ambient_props(altitide_cruise)[0]
C_L_c_w=(2*(W_s))/(rho_ac*(V_c**2))
print("Cruise lift coefficient =", C_L_c_w)
L_f = L_opt/l_L # Finding h-ho from trim equation -- needs to be refines
print ("Fuselage lenght = ", L_f , "m")
#X_appex = (-h_o*C_REF )+ (cg_p*L_f)+cg_d
h = (h_o*C_REF) - cg_d   
C_L_h_Opt = (C_mo_wf+(C_L_c_w*(h-h_o)))/(etha_h*V_H)# MOST IMPORTANT EQUATION IN TAIL DESIGN !!!! equation 6.29
print ("Horizontal tail desired lift coefficient = ",C_L_h_Opt)

Cruise lift coefficient = [0.97304657]
Fuselage lenght =  1.7262059165373125 m
Horizontal tail desired lift coefficient =  [-0.44658566]


In [89]:
AR_h = (2*AR/3)
Taper_h=0.6 # Initially this should match the wing taper ratio
Sweept_h=0.0000 # Initially this should match the wing sweept
C_l_alpha_h = 6.239746# data from airfoil
C_L_alpha = C_l_alpha_h/(1+(C_l_alpha_h/(pi*AR_h)))# Equation 6.57

alpha_h = np.rad2deg((C_L_h_Opt/C_l_alpha_h))
print("Tail angle of attack in cruise =",alpha_h,"degrees")



Tail angle of attack in cruise = [-4.1007236] degrees


9. Using thin airfoil theory check if tail is providing enough lift

In [90]:
# Data for employing the lifting line theory calculate the tail lift coefficient using the obtained angle of attack
print("Horizontal tail surface area =",S_h) 
print("Horizontal tail aspect ratio =",AR_h)
print("Horizontal tail aspect ratio =",Taper_h)
print("Airfoil lift curve slope",C_l_alpha_h )
print("Horixontal tail twist angle =",alpha_t)
print("Horizontal tail estimated angle of attack =",alpha_h)
print("Zero anlge of attack =",alpha_0)


Horizontal tail surface area = 0.12201830608447928
Horizontal tail aspect ratio = 6.0
Horizontal tail aspect ratio = 0.6
Airfoil lift curve slope 6.239746
Horixontal tail twist angle = 0.0
Horizontal tail estimated angle of attack = [-4.1007236]
Zero anlge of attack = 1e-06


In [91]:

b_h = np.sqrt(AR_h * S_h)  # Update the variable name
theta = np.linspace(np.pi / (2 * N), np.pi / 2, N)
alpha = alpha_h + alpha_t - np.linspace(0, alpha_t, N)
z = (b_h / 2) * np.cos(theta)
c = C_REF * (1 - (1 - Taper_h) * np.cos(theta))  # Mean Aerodynamic Chord at each segment
mu = c * C_l_alpha_h / (4 * b_h)
LHS = mu * (np.deg2rad(alpha) - np.deg2rad(alpha_0))
B = np.zeros((N, N))
A = np.zeros(N)
for i in range(N):
    for j in range(N):
        B[i, j] = np.sin((2 * j - 1) * theta[i]) * (1 + (mu[i] * (2 * j - 1)) / np.sin(theta[i]))
A = np.linalg.solve(B, LHS)
sum1 = np.zeros(N)
sum2 = np.zeros(N)

for i in range(N):
    for j in range(N):
        sum1[i] += (2 * j - 1) * A[j] * np.sin((2 * j - 1) * theta[i])
        sum2[i] += A[j] * np.sin((2 * j - 1) * theta[i])
CL_tail = np.pi * AR_h * A[0]  # Update the variable name
if abs(CL_tail)>  abs(C_L_h_Opt):
    print("Horizontal tail lift coefficient =", CL_tail, "& Required coefficient is", C_L_h_Opt, "So your tail is providing enough lift")
else:
    print("You are not providing enough lift; try changing your tail angle")


Horizontal tail lift coefficient = -0.6745421593484044 & Required coefficient is [-0.44658566] So your tail is providing enough lift


10. We now need to take into account the downwhas effects from the wing

In [92]:
e_o = (2*C_L_c_w)/(pi*AR)
deltha_e_deltha_a = (2*C_L_alpha)/(pi*AR)
e=e_o+(deltha_e_deltha_a*np.deg2rad(alpha_w))
i_h= alpha_h-alpha_f+np.rad2deg(e)


In [93]:
b_h = np.sqrt(AR_h * S_h)
C_h = b_h/AR_h
C_h_root=(3*C_h/2)/((1+Taper_h+(Taper_h**2))/(1+Taper_h))
C_h_tip = Taper_h* C_h_root
print("The final dimension of the Horizontal tail are:")
print("Spam = {0:1.5f}".format(b_h), "m")
print("MAC = {0:1.5f}".format(C_h), "m")
print("Root chord = {0:1.5f}".format(C_h_root), "m")
print("Tip chord = {0:1.5f}".format(C_h_tip), "m")
print("Surface area = {0:1.5f}".format(S_h), "m^2")
print("Aspect ratio = {0:1.5f}".format(AR_h), "")
print("Tail setting angle = {0:1.5f}".format(*i_h), "degrees")
print("Airfoil = S9032 (9%)")

The final dimension of the Horizontal tail are:
Spam = 0.85563 m
MAC = 0.14261 m
Root chord = 0.17462 m
Tip chord = 0.10477 m
Surface area = 0.12202 m^2
Aspect ratio = 6.00000 
Tail setting angle = 0.47283 degrees
Airfoil = S9032 (9%)


Vertical tail sizing using volume method
Data:

In [94]:
l_V = L_opt#Value taken from "optimum tail moment arm" in horizontal tail doc 
C_v_root=C_h_tip
print(C_v_root)

0.10477153128052936


In [95]:
S_V_total = (b_wing*S_REF*V_V)/l_V 
S_V_each = S_V_total/2
print("Each vertical Tail Planform Area =", S_V_each, "m^2")
C_v = C_v_root # since not taper will be used
C_v_tip = C_v_root
Taper_v = C_v_tip/C_v_root
b_v = S_V_each/C_v_root
AR_v = b_v/C_v

Each vertical Tail Planform Area = 0.03623899950214765 m^2


In [96]:
print("The final dimension of one vertical tail are:")
print("Spam = {0:1.5f}".format(b_v), "m")
print("MAC = {0:1.5f}".format(C_v), "m")
print("Root chord = {0:1.5f}".format(C_v_root), "m")
print("Tip chord = {0:1.5f}".format(C_v_tip), "m")
print("Surface area = {0:1.5f}".format(S_V_each), "m^2")
print("Aspect ratio = {0:1.5f}".format(AR_v), "")
print("Airfoil = S9032 (9%)")

The final dimension of one vertical tail are:
Spam = 0.34589 m
MAC = 0.10477 m
Root chord = 0.10477 m
Tip chord = 0.10477 m
Surface area = 0.03624 m^2
Aspect ratio = 3.30134 
Airfoil = S9032 (9%)
